In [1]:
%cd /alpha/projects/wastie/code/kondrashov/delta/notes

/alpha/projects/wastie/code/kondrashov/delta/notes


/alpha/projects/wastie/code/kondrashov/delta/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import sys
sys.path.append('../utils')
sys.path.append('..')

from utils.interpolate.markup_utils import Mask, load_markup, vis_markup, eternal_dataset_info, is_border_object
from utils.interpolate.refine_markup_by_yolo import mask_iou, mask_ioa, poly_mask_area

### Загрузка датасета и модели

In [3]:
CONFIG_PATH = '../config.json'
SPLIT = 'test'
IOU_THRESHOLD = 0.7

In [4]:
# Load config
import json
import numpy as np
from pathlib import Path

### Подготавливаем данные

In [5]:
# Create temporary directory for predictions
pred_labels_dir = Path('runs/segment/predict/labels')
pred_images_dir = Path('runs/segment/predict')

In [6]:
from utils.integrate.integrate import shift_mask

In [7]:
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

In [8]:
def restrict(obj):
    obj = obj.copy()
    obj['points'] = np.clip(obj['points'], 0, config['imgsz'])
    return obj

In [9]:
from tqdm import tqdm
from statistics import mode

def get_groups(markups, speed, imgsz):
    groups = [
        [-1] * len(m) for m in markups
    ]
    group_id = 0
    for i, markup in enumerate(tqdm(markups)):
        for j, obj in enumerate(markup):
            if groups[i][j] != -1:
                continue
            obj = restrict(obj)
            matched_groups = []
            # Find if group already exists
            idx = i-1
            shift = 0
            while idx >= 0 and shift < 0.5 * imgsz:
                shift += speed[idx + 1]
                for k, other_obj in enumerate(markups[idx]):
                    obj2 = restrict(shift_mask(other_obj, -shift))
                    if mask_ioa(obj, obj2) > IOU_THRESHOLD:
                        matched_groups.append(groups[idx][k])
                idx -= 1
            
            if len(matched_groups) > 2:
                groups[i][j] = mode(matched_groups)
            else:
                groups[i][j] = group_id
                group_id += 1

            # Find next elements from this group
            idx = i+1
            shift = 0
            while idx < len(markups) and shift < 1.2 * imgsz:
                shift += speed[idx]
                for k, other_obj in enumerate(markups[idx]):
                    if groups[idx][k] != -1:
                        continue
                    if mask_ioa(obj, restrict(shift_mask(other_obj, shift))) > IOU_THRESHOLD:
                        groups[idx][k] = groups[i][j]
                idx += 1
    return groups

In [10]:
OLD_CLS_TO_NEW_INDICES = {'0': '0', '1': '0', '2': '1'}
def cls_transform(markup):
    result = []
    for m in markup.copy():
        m['cls'] = OLD_CLS_TO_NEW_INDICES[m['cls']]
        result.append(m)
    return result

In [11]:
from tqdm import tqdm
def get_matches(markups, other):
    is_matched = [
        [False] * len(m) for m in markups
    ]
    is_mask_matched = [
        [False] * len(m) for m in markups
    ]
    is_ioa_matched = [
        [False] * len(m) for m in markups
    ]
    is_mask_ioa_matched = [
        [False] * len(m) for m in markups
    ]
    possible_cls = [
        [set()] * len(m) for m in markups
    ]
    for i, markup in enumerate(tqdm(markups)):
        for j, obj in enumerate(markup):
            obj = restrict(obj)
            for other_obj in other[i]:
                other_obj = restrict(other_obj)
                # Match mask
                if mask_iou(obj, other_obj) > IOU_THRESHOLD:
                    is_mask_matched[i][j] = True
                    # Match class
                    if obj['cls'] == other_obj['cls']:
                        is_matched[i][j] = True
                    else:
                        possible_cls[i][j].add(other_obj['cls'])
                # if mask_ioa(obj, other_obj) > IOU_THRESHOLD:
                #     is_mask_ioa_matched[i][j] = True
                #     # Match class
                #     if obj['cls'] == other_obj['cls']:
                #         is_ioa_matched[i][j] = True
                    
    return is_matched, is_mask_matched, is_ioa_matched, is_mask_ioa_matched, possible_cls

In [12]:
def idx_to_groups(group_idx, matching_info, markup, img_paths):
    is_matched, is_mask_matched, is_ioa_matched, is_mask_ioa_matched, possible_cls = matching_info
    groups = [[] for i in range(max(y for x in group_idx for y in x) + 1)]
    for i, group in enumerate(group_idx):
        for j, g in enumerate(group):
            img_path = img_paths[i]
            groups[g].append({
                "img": img_path, 
                "obj": markup[i][j], 
                "is_border": is_border_object(Mask(markup[i][j]), (config['imgsz'], config['imgsz'])), 
                "is_matched": is_matched[i][j],
                "is_mask_matched": is_mask_matched[i][j],
                "is_ioa_matched": is_ioa_matched[i][j],
                "is_mask_ioa_matched": is_mask_ioa_matched[i][j],
                "possible_cls": possible_cls[i][j],
            })
    return groups

### Предсказание с лучшим по F1 confidence

In [13]:
MODEL_VERSION = 'noborder_aug'

In [14]:
import shutil

def find_best_conf(model, config):
    # Run validation to get best confidence threshold
    val_results = model.val(data=config['data'], split=SPLIT, device='cuda:1')

    best_f1_idx = np.argmax(val_results.seg.curves_results[1][1].mean(axis=0))
    best_f1 = val_results.seg.curves_results[1][1][..., best_f1_idx].mean()
    best_conf = val_results.seg.curves_results[1][0][best_f1_idx]
    print(f"Best F1: {best_f1:.4f} at confidence {best_conf:.4f}")
    return best_conf


def get_gt_and_predicted_groups(model, config, dataset_path, conf):

    # Load labels
    dataset_path = Path(dataset_path)
    dataset_info = eternal_dataset_info(dataset_path)
    gt_labels_dir = Path(dataset_path) / 'gt_interp'
    
    shutil.rmtree('runs/segment', ignore_errors=True)
    # Run YOLO validation to get the best confidence score

    # Run prediction with best confidence
    for pred in model.predict(
        source=str(dataset_path / 'imgs'),
        conf=conf,
        save_txt=True,
        device='cuda:1',
        save=True,
        stream=True,
        save_conf=True,
    ):
        pass
    
    gt = []
    pred = []
    speed = []
    gt_paths = sorted(list(gt_labels_dir.glob("*.txt")))
    for gt_path in gt_paths:
        pred_path = pred_labels_dir / gt_path.name
        if not pred_path.exists():
            pred_path.touch()
        gt.append(cls_transform(load_markup(gt_path, config['imgsz'])))
        pred.append(load_markup(pred_path, config['imgsz']))
        speed.append(dataset_info['speed'][str(dataset_path / 'imgs' / (gt_path.stem + '.jpg'))])

    gt_img_paths = [p.parent.parent / 'imgs' / f'{p.stem}.jpg' for p in gt_paths]
    pred_img_paths = [pred_images_dir / f'{p.stem}.jpg' for p in gt_paths]

    gt_group_idx = get_groups(gt, speed, config['imgsz'])
    gt_matching_info = get_matches(gt, pred)
    gt_groups = idx_to_groups(gt_group_idx, gt_matching_info, gt, gt_img_paths)
    
    pred_group_idx = get_groups(pred, speed, config['imgsz'])
    pred_matching_info = get_matches(pred, gt)
    pred_groups = idx_to_groups(pred_group_idx, pred_matching_info, pred, pred_img_paths)
    
    return gt_groups, pred_groups

In [15]:
# gt_groups_per_dataset = []
# pred_groups_per_dataset = []

# from ultralytics import YOLO
# model = YOLO(config['models'][MODEL_VERSION])
# import torch

# conf = find_best_conf(model, config)

# for dataset_path in config['interpolated']['datasets']:
#     gt_groups, pred_groups = get_gt_and_predicted_groups(model, config, dataset_path, conf)
#     gt_groups_per_dataset.append(gt_groups)
#     pred_groups_per_dataset.append(pred_groups)

In [ ]:
CHOSEN_DATASETS = config['interpolated']['datasets']['train']

In [18]:
from shutil import copy

In [ ]:
gt_groups_per_dataset = []
pred_groups_per_dataset = []

from ultralytics import YOLO
model = YOLO('/alpha/projects/wastie/code/kondrashov/tmp/dense_models/24_12.pt')
import torch

conf = find_best_conf(model, config)
#conf = 0.6

for dataset_path in CHOSEN_DATASETS:
    # gt_groups, pred_groups = get_gt_and_predicted_groups(model, config, dataset_path, conf)
    # gt_groups_per_dataset.append(gt_groups)
    # pred_groups_per_dataset.append(pred_groups)
    for img_path in (Path(dataset_path) / 'imgs').iterdir():
        copy(img_path, '/alpha/projects/wastie/code/kondrashov/delta/data/trainval/imgs')

Ultralytics 8.3.48 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:1 (NVIDIA A100 80GB PCIe, 81154MiB)
YOLOv8m-seg summary (fused): 263 layers, 24,586,614 parameters, 0 gradients, 98.7 GFLOPs


val: Scanning /alpha/projects/wastie/datasets/25_12_2_classes_fpfn_update_v2/test/labels.cache... 731 images, 150 backgrounds, 0 corrupt: 100%|██████████| 731/731 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:10<00:00,  4.49it/s]


                   all        731       4144      0.925      0.886      0.952      0.843      0.926      0.887      0.951      0.788
                   bot        540       3093      0.914      0.866      0.946      0.836      0.916      0.868      0.945      0.784
                  alum        428       1051      0.936      0.906      0.957      0.849      0.936      0.906      0.957      0.792
Speed: 0.7ms preprocess, 4.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/segment/val
Best F1: 0.9067 at confidence 0.4555


KeyboardInterrupt: 

In [ ]:
# Update border objects and calculate area
for g in gt_groups:
    for instance in g:
        instance['is_border'] = is_border_object(Mask(instance['obj']), (config['imgsz'], config['imgsz']))
        instance['area'] = poly_mask_area(instance['obj'])
for g in pred_groups:
    for instance in g:
        instance['is_border'] = is_border_object(Mask(instance['obj']), (config['imgsz'], config['imgsz']))
        instance['area'] = poly_mask_area(instance['obj'])

In [ ]:
# Calculate area share based on the group's max area
for groupset in [gt_groups, pred_groups]:
    for g in groupset:
        max_area = max([instance['area'] for instance in g])
        for instance in g:
            instance['area_share'] = instance['area'] / max_area

In [ ]:
import pickle
with open('gt_24_12_dense_c06.pkl', 'wb') as f:
    pickle.dump(gt_groups, f)
with open('pred_24_12_dense_c06.pkl', 'wb') as f:
    pickle.dump(pred_groups, f)

In [ ]:
# import pickle
# with open('gt_groups_test_extended.pkl', 'wb') as f:
#     pickle.dump(gt_groups, f)
# with open('pred_groups_test_extended.pkl', 'wb') as f:
#     pickle.dump(pred_groups, f)
# with open('gt_groups_per_dataset_trainval.pkl', 'wb') as f:
#     pickle.dump(gt_groups_per_dataset, f)
# with open('pred_groups_per_dataset_trainval.pkl', 'wb') as f:
#     pickle.dump(pred_groups_per_dataset, f)

## Генерация общего датасета

In [ ]:
import pickle
# with open('gt_groups.pkl', 'rb') as f:
#     gt_groups = pickle.load(f)
# with open('pred_groups.pkl', 'rb') as f:
#     pred_groups = pickle.load(f)
with open('gt_groups_per_dataset_trainval.pkl', 'rb') as f:
    gt_groups_per_dataset = pickle.load(f)
with open('pred_groups_per_dataset_trainval.pkl', 'rb') as f:
    pred_groups_per_dataset = pickle.load(f)

gt_groups = []
pred_groups = []
for gt_group in gt_groups_per_dataset:
    gt_groups += gt_group
for pred_group in pred_groups_per_dataset:
    pred_groups += pred_group

FileNotFoundError: [Errno 2] No such file or directory: 'gt_groups_per_dataset_trainval.pkl'

In [ ]:
gt_groups = []
pred_groups = []
for gt_group in gt_groups_per_dataset:
    gt_groups += gt_group
for pred_group in pred_groups_per_dataset:
    pred_groups += pred_group

In [ ]:
len(pred_groups)

12333

In [ ]:
import pickle
with open('gt_test_dense.pkl', 'rb') as f:
    gt_groups = pickle.load(f)
with open('pred_test_dense.pkl', 'rb') as f:
    pred_groups = pickle.load(f)

##### Отфильтурем объекты по тем, что встречались в неинтерполированном тестовом датасете

In [ ]:
import random
interp_test_imgs_path = Path(config['interpolated']['test']).parent / 'test' / 'images'
#interpolated_test_names = set(p.name for p in interp_test_imgs_path.iterdir() if p.name.startswith('tula_sep_0002_2024_07_22_18'))
interpolated_test_names = set(p.name for p in interp_test_imgs_path.iterdir())

In [ ]:
gt_groups = [[m for m in g if m['img'].name in interpolated_test_names] for g in gt_groups]
gt_groups = [g for g in gt_groups if len(g) > 0]

In [ ]:
pred_groups = [[m for m in g if m['img'].name in interpolated_test_names] for g in pred_groups]
pred_groups = [g for g in pred_groups if len(g) > 0]

In [ ]:
cnt = {}
for g in pred_groups:
    for m in g:
        dir = m['img'].name[:len('tula_sep_0002_2024_07_22_18')]
        if dir not in cnt:
            cnt[dir] = 0
        cnt[dir] += 1
for k, v in cnt.items():
    print(k, '\t', v)

tula_sep_0002_2024_07_16_14 	 8994
tula_sep_0002_2024_07_16_15 	 8221


In [ ]:
cnt = {}
for g in gt_groups:
    for m in g:
        dir = m['img'].name[:len('tula_sep_0002_2024_07_22_18')]
        if dir not in cnt:
            cnt[dir] = 0
        cnt[dir] += 1
for k, v in cnt.items():
    print(k, '\t', v)

tula_sep_0002_2024_07_16_14 	 10935
tula_sep_0002_2024_07_16_15 	 9748


##### Добавим дополнительные свойства

In [ ]:
# Update border objects and calculate area
for g in gt_groups:
    for instance in g:
        instance['is_border'] = is_border_object(Mask(instance['obj']), (config['imgsz'], config['imgsz']))
        instance['area'] = poly_mask_area(instance['obj'])
for g in pred_groups:
    for instance in g:
        instance['is_border'] = is_border_object(Mask(instance['obj']), (config['imgsz'], config['imgsz']))
        instance['area'] = poly_mask_area(instance['obj'])

In [ ]:
# Calculate area share based on the group's max area
for groupset in [gt_groups, pred_groups]:
    for g in groupset:
        max_area = max([instance['area'] for instance in g])
        for instance in g:
            instance['area_share'] = instance['area'] / max_area